In [9]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
import re
import folium
import geopandas as gpd
from datetime import datetime as dt
import numpy as np
from branca.element import Template, MacroElement

In [10]:
raw_data=r.get("https://prokazan-ru.turbopages.org/prokazan.ru/s/news/view/142788").text
soup=BeautifulSoup(raw_data)
distrs=soup.find_all("p",{"class":"paragraph unit unit_text_m"})[2:5]
megadict={}


for distr in distrs:
    regs=re.findall(r"\d+\)\s?[а-я–\-А-Я]+\s?[–-]\s?\d+",distr.text)
    cases=[re.search(r"\d+\)\s?[а-я–\-А-Я]+\s?[–-]\s?(\d+)",i).group(1) for i in regs]
    names=[re.search(r"\d+\)\s?([а-я–\-А-Я]+)\s?[–-]\s?(\d+)",i).group(1) for i in regs]
    for ind,val in enumerate(names):
        megadict[val]=cases[ind]

megadict.update({"Казань":6597})
megadict.update({"Набережные Челны":1207})
data=pd.DataFrame([megadict.keys(),megadict.values()]).T
data.columns=["Район","Случаи"]
data["Случаи"]=data["Случаи"].astype("int")
data.to_excel(r"C:/Users/timna/Covid_WebSite/Data/Covid19_17.12.2020.xlsx",index=False)

In [11]:
geo_data=gpd.read_file(r"C:/Users/timna/Russia_geojson_OSM\GeoJson's\Regions\PFO\Республика Татарстан_Tatarstan.geojson")
covid_data=pd.read_excel("C:/Users/timna/Covid_WebSite/Data/Covid19_17.12.2020.xlsx")

In [12]:
covid_data["Район"]=[i+" район" if i not in ["Казань","Набережные Челны"] else "городской округ "+i for i in covid_data["Район"]]

In [13]:
final_data=geo_data.merge(covid_data,right_on='Район',left_on='district')[["district","geometry","Случаи"]]

In [14]:
m=folium.Map([55.4152363,50.8576256],zoom_start=7,zoom_control=False,
               scrollWheelZoom=False,
                max_zoom=7,
                min_zoom=7,
               dragging=True)

In [15]:
q=list(np.quantile(final_data["Случаи"],[0.25,0.5,0.75,0.98]))

In [16]:
q

[29.0, 61.0, 157.0, 1853.799999999986]

In [17]:
palette=[]
for val in final_data.itertuples():
    if val[-1]<q[0]:
        palette.append("#faa307")
    elif val[-1]>=q[0] and val[-1]<q[1]:
        palette.append("#e85d04")
    elif val[-1]>=q[1] and val[-1]<q[2]:
        palette.append("#d00000")
    elif val[-1]>=q[2] and val[-1]<q[3]:
        palette.append("#6a040f")
    elif val[-1]>=q[3]:
        palette.append("#03071e")


final_data["Color"]=palette

folium.GeoJson(final_data[["district","geometry","Случаи","Color"]].to_json(),style_function=lambda x: {'fillColor':x["properties"]["Color"],
                                                         'fillOpacity': 0.9, 'opacity': 0.7},
                           highlight_function=lambda x: {"fillOpacity": 0.3, "opacity": 0.3},
                           tooltip=folium.features.GeoJsonTooltip(fields=["district","Случаи"],aliases=["Район:","Случаев:"])).add_to(m)

a = """
        {% macro html(this, kwargs) %}

        <!doctype html>
        <html lang="en">
        <head>
          <meta charset="utf-8">
          <meta name="viewport" content="width=device-width, initial-scale=1">
          <title>Карта</title>
          <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

          <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
          <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

          <script>
          $( function() {
            $( "#maplegend" ).draggable({
                            start: function (event, ui) {
                                $(this).css({
                                    right: "auto",
                                    top: "auto",
                                    bottom: "auto"
                                });
                            }
                        });
        });

          </script>
        </head>
        <body>


        <div id='maplegend' class='maplegend' 
            style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
             border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>"""

b = """<div class='legend-title'>Случаи</div>
        <div class='legend-scale'>
          <ul class='legend-labels'>
            <li><span style='background:#faa307;opacity:0.7;'></span><{}</li>
            <li><span style='background:#e85d04;opacity:0.7;'></span>{}< и <{}</li>
            <li><span style='background:#d00000;opacity:0.7;'></span>{}< и <{}</li>
            <li><span style='background:#6a040f;opacity:0.7;'></span>{}< и <{}</li>
            <li><span style='background:#03071e;opacity:0.7;'></span>>{}</li>
          </ul>
        </div>
        </div>""".format(np.round(q[0]),np.round(q[0]),np.round(q[1]),np.round(q[1]),np.round(q[2]),np.round(q[2]),np.round(q[3]),np.round(q[3]))

z = """</body>
        </html>

        <style type='text/css'>
          .maplegend .legend-title {
            text-align: left;
            margin-bottom: 5px;
            font-weight: bold;
            font-size: 90%;
            }
          .maplegend .legend-scale ul {
            margin: 0;
            margin-bottom: 5px;
            padding: 0;
            float: left;
            list-style: none;
            }
          .maplegend .legend-scale ul li {
            font-size: 80%;
            list-style: none;
            margin-left: 0;
            line-height: 18px;
            margin-bottom: 2px;
            }
          .maplegend ul.legend-labels li span {
            display: block;
            float: left;
            height: 16px;
            width: 30px;
            margin-right: 5px;
            margin-left: 0;
            border: 1px solid #999;
            }
          .maplegend .legend-source {
            font-size: 80%;
            color: #777;
            clear: both;
            }
          .maplegend a {
            color: #777;
            }
        </style>
        {% endmacro %}"""
macro = MacroElement()
macro._template = Template(a + b + z)
m.add_child(macro)
title_html = '''
             <h3 align="center" style="font-size:19px"><b>Данные о распространении коронавируса в Республике Татарстан за {}</b></h3>
             '''.format("17.12.2020")
m.get_root().html.add_child(folium.Element(title_html))

In [30]:
m.save("C:/Users/timna/Covid_WebSite/Maps/index_17.12.2020.html")

In [18]:
m